In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import radians, cos, sin, asin, sqrt
import os

# showing all columns
pd.set_option('display.max_columns', None)

df = pd.read_csv('../data/raw/amazon_delivery.csv')

df['Order_Date'] = df['Order_Date'].astype(str)
df['Order_Time'] = df['Order_Time'].astype(str)
df['Pickup_Time'] = df['Pickup_Time'].astype(str)

# removing rows without order or pickup timeد
df = df[~df['Order_Time'].str.lower().str.contains('nan')]
df = df[~df['Pickup_Time'].str.lower().str.contains('nan')]

# making columns with date + time
df['Order_Datetime'] = pd.to_datetime(df['Order_Date'] + ' ' + df['Order_Time'], errors='coerce')
df['Pickup_Datetime'] = pd.to_datetime(df['Order_Date'] + ' ' + df['Pickup_Time'], errors='coerce')
df['Order_Date'] = pd.to_datetime(df['Order_Date'], errors='coerce')

# removing wrong rows
df = df.dropna(subset=['Order_Datetime', 'Pickup_Datetime'])

# delay of rider
df['Pickup_Delay_Minutes'] = (df['Pickup_Datetime'] - df['Order_Datetime']).dt.total_seconds() / 60

# haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    return 2 * R * asin(sqrt(a))

# distance for every row
df['Distance_KM'] = df.apply(lambda row: haversine(row['Store_Latitude'], row['Store_Longitude'],
                                                   row['Drop_Latitude'], row['Drop_Longitude']), axis=1)

# for not division to 0
df = df[df['Delivery_Time'] > 0]
df = df[df['Pickup_Datetime'] >= df['Order_Datetime']]

# speed of rider
df['Delivery_Speed_KMPH'] = df['Distance_KM'] / (df['Delivery_Time'] / 60)

# removing wrong distances
df = df[df['Pickup_Datetime'] >= df['Order_Datetime']]
df = df[df['Distance_KM'] <= 500]
df = df[df['Delivery_Speed_KMPH'] <= 150]

# replacing non in agent rating and weather
df['Agent_Rating'] = df['Agent_Rating'].fillna(df['Agent_Rating'].mean())
df['Weather'] = df['Weather'].fillna('Unknown')

# final file
os.makedirs('../data/processed', exist_ok=True)
df.to_csv('../data/processed/cleaned_amazon.csv', index=False)

# key results
print(df[['Pickup_Delay_Minutes', 'Distance_KM', 'Delivery_Speed_KMPH']].describe())


       Pickup_Delay_Minutes   Distance_KM  Delivery_Speed_KMPH
count          42666.000000  42666.000000         42666.000000
mean               9.958046      9.690584             5.879556
std                4.085759      5.610145             6.626779
min                5.000000      1.465067             0.339202
25%                5.000000      4.658009             2.602814
50%               10.000000      9.193028             4.291371
75%               15.000000     13.680927             6.740944
max               15.000000     20.969489           119.850915
